# Phase 4: 객체 인식

이 노트북에서 배울 내용:
- 기하학적 특징 추출
- FPFH 특징 추출
- 규칙 기반 분류
- 머신러닝 분류기 (SVM)

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import open3d as o3d

from src.data import PointCloudLoader
from src.preprocessing import Preprocessor
from src.segmentation import Segmenter
from src.recognition import PointCloudClassifier, FeatureExtractor
from src.recognition.classifier import SimpleRuleClassifier
from src.utils import Visualizer

## 1. 학습 데이터 생성

In [ ]:
loader = PointCloudLoader()

# 각 클래스별 샘플 생성
def generate_samples(n_samples=20):
    """학습용 샘플 생성"""
    samples = []
    labels = []
    
    for i in range(n_samples):
        # 박스 (다양한 크기)
        size = np.random.uniform(0.3, 1.0)
        box = loader.create_sample_box(
            center=(0, 0, 0),
            size=(size, size * np.random.uniform(0.8, 1.2), size * np.random.uniform(0.8, 1.2)),
            density=500
        )
        samples.append(box)
        labels.append("box")
        
        # 구 (다양한 크기)
        radius = np.random.uniform(0.2, 0.6)
        sphere = loader.create_sample_sphere(
            center=(0, 0, 0),
            radius=radius,
            n_points=500
        )
        samples.append(sphere)
        labels.append("sphere")
    
    return samples, labels

samples, labels = generate_samples(20)
print(f"생성된 샘플: {len(samples)}개")
print(f"클래스: {set(labels)}")

## 2. 기하학적 특징 추출

In [ ]:
feature_extractor = FeatureExtractor()

# 박스 특징
box_features = feature_extractor.extract_geometric_features(samples[0])
print("박스 특징:")
for key, value in box_features.items():
    if isinstance(value, list):
        print(f"  {key}: [{value[0]:.3f}, {value[1]:.3f}, {value[2]:.3f}]")
    else:
        print(f"  {key}: {value:.3f}")

In [ ]:
# 구 특징
sphere_features = feature_extractor.extract_geometric_features(samples[1])
print("구 특징:")
for key, value in sphere_features.items():
    if isinstance(value, list):
        print(f"  {key}: [{value[0]:.3f}, {value[1]:.3f}, {value[2]:.3f}]")
    else:
        print(f"  {key}: {value:.3f}")

In [ ]:
# 핵심 특징 비교
print("\n=== 형상 특징 비교 ===")
print(f"               박스      구")
print(f"Linearity:    {box_features['linearity']:.3f}   {sphere_features['linearity']:.3f}")
print(f"Planarity:    {box_features['planarity']:.3f}   {sphere_features['planarity']:.3f}")
print(f"Sphericity:   {box_features['sphericity']:.3f}   {sphere_features['sphericity']:.3f}")

# 구는 sphericity가 높고, 박스는 planarity가 높은 경향

## 3. FPFH 특징 추출

In [ ]:
# FPFH: 33차원 히스토그램 특징
fpfh_box = feature_extractor.extract_global_fpfh(samples[0])
fpfh_sphere = feature_extractor.extract_global_fpfh(samples[1])

print(f"FPFH 차원: {len(fpfh_box)}")
print(f"박스 FPFH (처음 5개): {fpfh_box[:5]}")
print(f"구 FPFH (처음 5개): {fpfh_sphere[:5]}")

In [ ]:
# FPFH 시각화
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.bar(range(33), fpfh_box)
plt.title('Box FPFH')
plt.xlabel('Bin')

plt.subplot(1, 2, 2)
plt.bar(range(33), fpfh_sphere)
plt.title('Sphere FPFH')
plt.xlabel('Bin')

plt.tight_layout()
plt.show()

## 4. 규칙 기반 분류

In [ ]:
# 규칙 기반 분류기 (학습 불필요)
rule_classifier = SimpleRuleClassifier()

# 테스트
for i, (sample, label) in enumerate(zip(samples[:4], labels[:4])):
    pred, scores = rule_classifier.classify(sample)
    print(f"샘플 {i+1} (실제: {label}): 예측 = {pred}")
    print(f"  점수: {scores}")
    print()

## 5. 머신러닝 분류기 (SVM)

In [ ]:
# 데이터 분할
from sklearn.model_selection import train_test_split

train_samples, test_samples, train_labels, test_labels = train_test_split(
    samples, labels, test_size=0.2, random_state=42
)

print(f"학습 데이터: {len(train_samples)}개")
print(f"테스트 데이터: {len(test_samples)}개")

In [ ]:
# SVM 분류기 학습
classifier = PointCloudClassifier(classifier_type="svm")
train_result = classifier.train(train_samples, train_labels)

print("\n학습 결과:")
print(f"  학습 정확도: {train_result['train_accuracy']:.2%}")
print(f"  클래스: {train_result['classes']}")

In [ ]:
# 테스트
correct = 0
for sample, label in zip(test_samples, test_labels):
    pred, probs = classifier.predict(sample)
    is_correct = pred == label
    correct += is_correct
    
    status = "✓" if is_correct else "✗"
    print(f"{status} 실제: {label:6s} | 예측: {pred:6s} | 확률: {probs}")

print(f"\n테스트 정확도: {correct}/{len(test_samples)} = {correct/len(test_samples):.2%}")

## 6. 모델 저장/로드

In [ ]:
# 모델 저장
classifier.save("../models/shape_classifier.pkl")

# 새 분류기로 로드
new_classifier = PointCloudClassifier()
new_classifier.load("../models/shape_classifier.pkl")

# 로드된 모델로 예측
pred, probs = new_classifier.predict(test_samples[0])
print(f"예측: {pred}, 확률: {probs}")

## 7. 실제 씬에서 객체 분류

In [ ]:
# 씬 생성 및 세그멘테이션
preprocessor = Preprocessor()
segmenter = Segmenter()

scene = loader.create_sample_scene(add_noise=True)
scene = preprocessor.full_pipeline(scene, voxel_size=0.03)
planes, clusters = segmenter.segment_scene(scene)

In [ ]:
# 각 클러스터 분류
print("\n=== 씬 내 객체 분류 ===")
for i, cluster in enumerate(clusters):
    # 정규화
    normalized = FeatureExtractor.normalize_point_cloud(cluster.points)
    
    # 규칙 기반 분류
    pred, scores = rule_classifier.classify(cluster.points)
    
    bbox_size = cluster.bbox.get_extent()
    print(f"클러스터 {i+1}:")
    print(f"  크기: {bbox_size[0]:.2f} x {bbox_size[1]:.2f} x {bbox_size[2]:.2f}")
    print(f"  예측: {pred}")
    print()

## 연습 문제

1. 원기둥(cylinder) 클래스를 추가하고 분류기를 재학습해보세요.
2. Random Forest 분류기로 바꿔서 성능을 비교해보세요.
3. 학습 데이터 수를 늘려 정확도 변화를 확인해보세요.

In [ ]:
# 연습 코드 작성


## 다음 단계: PointNet

전통적인 방법의 한계:
- 수작업으로 설계한 특징에 의존
- 복잡한 형상 인식에 한계

**PointNet**은 Point Cloud를 직접 입력받아 특징을 자동 학습합니다.

다음 학습 자료:
- 논문: "PointNet: Deep Learning on Point Sets" (Qi et al., 2017)
- 데이터셋: ModelNet40